In [1]:
#!/usr/bin/python
# with positive feedback
# Modified 11/29/2019 by HYK
# signal-induced repression model 
# Incorporate binomial partitioning, DONE!
# Cumulative addition of time over multiple rounds of simulation, DONE!

# Generalized functions for simulating arbitrary reaction network topologies
# Simulation cell number distributions as well, for multiple runs
# Also, write a MATLAB script to rapidly generate plots from the simulated data

"""
Created on Tue Dec 13 20:08:54 2016
@author: Sam Nguyen
"""
import roadrunner
import numpy 
from numpy import random as rand
from numpy import nonzero as nonzero
import math
from scipy import asarray as ar,exp
from multiprocessing import Pool
import csv
import random
import sys
roadrunner.Config.setValue(roadrunner.Config.MAX_OUTPUT_ROWS,10000)

# Tellurium loader
def loada(sbstr):
    import antimony as sb
    r = sb.loadAntimonyString(sbstr)
    if r < 0:
        raise RuntimeError('Failed to load Antimony model: {}'.format(sb.getLastError()))
    return roadrunner.RoadRunner(sb.getSBMLString(sb.getModuleNames()[-1]))

# Tellurium simulation function
def doSimulation(tc,nc,S):
    r = loada("""
    # define cell cycling parameters
    TCycle = 10                                    
    V := (time/TCycle + 1)
    
    #define ode
    # should we include positive feedback additively
    R1: => X ; V*(S*alphaX0*Kx/(Kx+S))  # X is synthesized but repressed b 
    R2: X => ; X*deltaX             # degradation of X
    R3: => Y ; (alphaY*V)*(X/V)^N/((X/V)^N+Ky^N)
    R4: Y => ; Y*deltaY
   
   #define initial values
    S = 1.0             # the concentration of signal * (normalized units) 
    X = 0                 # the initial number of copies of X
    Y = 0                 # the initial number of copies of Y
 
    alphaX0 = 5         # synthesis rate of X (copies/hr)
    alphaX1 = 70
    deltaX = 0             # degradation rate (1/hr), assume infinitely stable
    alphaY = 10^3          # synthesis rate of Y (copies/hr)
    deltaY = 1             # degradation rate (1/hr)
    Kx = 10
    Ky = 1000              # MM constant for activation of Y by X
    M = 15
    N = 10                 # Hill coefficient for activation of Y by X
    """)
    # set parameters of stochastic simulation
    r.integrator = 'gillespie'
    r.integrator.variable_step_size = False  
    r.TCycle = tc # define cell cycle time 
    
    ############################
    r.S = float(S)    # this is the signal level
    dt = 0.1;     # time interval for plotting
    result=[]
    threshold = 500   # threshold of Y required for activation
    for i in range(0,nc,1):       # for loop over the total number of cell cycles
        if i == 0:
            SubResult =r.simulate(0,tc,int(tc/dt),['time','V','X','Y'])
            for row in SubResult:
                result.append(row)
        else:
            r.X = rand.binomial(r.X, 0.5)   # randomly partition
            r.Y = rand.binomial(r.Y, 0.5)
            SubResult =r.simulate(0,tc,int(tc/dt),['time','V','X','Y'])             
            for row in SubResult:
                row[0]=row[0]+(i*tc)     # update the time vector
                result.append(row)
               
        result2 = ar(result)
        yc = result2[:,3]/result2[:,1]    # concentration
        yt = (yc>threshold)   # crossing the concentration threshold
        tt = nonzero(yt)      # transition time
        if (len(tt[0])>0):     # a transition occurred, 
            tt = tt[0][0]
            act_time = result2[tt,0]   # find the transition time
            #print('transition detected at {}'.format(act_time))
            return act_time
        
    act_time = -1   # no transition detected at the end of the simulation
    return act_time

#Write csv file
def writeCSV(filename,result):
    with open(filename,'w') as mycvsfile:
        thedatawriter=csv.writer(mycvsfile)   
        thedatawriter.writerows(result)


In [ ]:
datapoints = [25, 12, 8, 4, 2, 1.5, 1.0, 0.9, 0.85, 0.80, 0.75]

# old 25, 12, 8, 4, 2, 1.4, 1.2, 1.0, 0.95, 0.9, 0.87, 0.84, 0.8
for i in datapoints:    # titrate in a log scale
    S = i
    print('parameter value S={}'.format(i))
    act_times = []
    for j in range(0,100,1):
        act_time = doSimulation(10,int(25*2/S),S)
        act_times.append(act_time)       # find the transition time
        print('repeat {}, activation time ={}'.format(j,act_time))  
    with open('ttimes_{}.csv'.format(i),'w') as mycvsfile:
        thedatawriter=csv.writer(mycvsfile)   
        thedatawriter.writerows(map(lambda x: [x], act_times))